# se3

> SE(3) exp and log map

In [ ]:
#| default_exp se3

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch

from pytorchse3.so3 import cross_product_matrix, so3_log_map

## Log map

In [ ]:
#| exporti
"""
Taylor expansions taken from : https://github.com/SYSU-STAR/H2-Mapping/blob/11b8ab15f3302ccb2b4b3d2b30f76d86dcfcde2c/mapping/src/se3pose.py#L89-L118
"""

def taylor_A(x, nth=10):
    # Taylor expansion of sin(x)/x
    ans = torch.zeros_like(x)
    denom = 1.0
    for i in range(nth + 1):
        if i > 0:
            denom *= (2 * i) * (2 * i + 1)
        ans = ans + (-1) ** i * x ** (2 * i) / denom
    return ans


def taylor_B(x, nth=10):
    # Taylor expansion of (1-cos(x))/x**2
    ans = torch.zeros_like(x)
    denom = 1.0
    for i in range(nth + 1):
        denom *= (2 * i + 1) * (2 * i + 2)
        ans = ans + (-1) ** i * x ** (2 * i) / denom
    return ans


def taylor_C(x, nth=10):
    # Taylor expansion of (x-sin(x))/x**3
    ans = torch.zeros_like(x)
    denom = 1.0
    for i in range(nth + 1):
        denom *= (2 * i + 2) * (2 * i + 3)
        ans = ans + (-1) ** i * x ** (2 * i) / denom
    return ans

In [ ]:
#| export
def se3_log_map(T: torch.Tensor):
    R = T[..., :3, :3]
    t = T[..., :3, 3]

    log_R_vee = so3_log_map(R)
    log_R = cross_product_matrix(log_R_vee)
    log_R_2 = torch.linalg.matrix_power(log_R, 2)
    
    theta = log_R_vee.norm(dim=-1, keepdim=True).unsqueeze(-1)
    A = taylor_A(theta)
    B = taylor_B(theta)
    D = (1 - A / (2 * B)) / theta.pow(2)
    
    V_inv = torch.eye(3, dtype=A.dtype, device=A.device) - 0.5 * log_R + D * log_R_2
    log_t_vee = torch.einsum("bij, bj -> bi", V_inv, t)

    return torch.concat([log_R_vee, log_t_vee], dim=-1)

## Exp map

In [ ]:
#| export
def se3_exp_map(log_T_vee, n=10):
    log_R_vee = log_T_vee[..., :3]
    log_t_vee = log_T_vee[..., 3:]

    theta = log_R_vee.norm(dim=-1, keepdim=True).unsqueeze(-1)
    log_R = cross_product_matrix(log_R_vee)
    log_R_2 = torch.linalg.matrix_power(log_R, 2)
    
    A = taylor_A(theta, n)
    B = taylor_B(theta, n)
    C = taylor_C(theta, n)
    
    R = torch.eye(3, dtype=A.dtype, device=A.device) + A * log_R + B * log_R_2
    V = torch.eye(3, dtype=A.dtype, device=A.device) + B * log_R + C * log_R_2
    t = torch.einsum("bij, bj -> bi", V, log_t_vee)
    
    T = torch.zeros((len(theta), 4, 4), dtype=A.dtype, device=A.device)
    T[..., :3, :3] = R
    T[..., :3, 3] = t
    T[..., 3, 3] = 1.0
    return T

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()